In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import (
    FewShotPromptTemplate
)
import pickle
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
#model_id ='meta-llama/Meta-Llama-3.1-8B-Instruct'

def get_embedding():
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_and_vectorize(csv_path, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    # CSV 파일을 불러와 데이터프레임 생성
    train_df = pd.read_csv(csv_path)
    train_df.drop('SAMPLE_ID', axis=1, inplace=True)
    trainset = train_df.to_dict(orient='records')
    print("Loaded Fewshot Set:", trainset[:3])
    
    # 벡터화할 텍스트 생성
    to_vectorize = ["\n\n".join(example.values()) for example in trainset]
    
    # 벡터화 및 FAISS DB 생성
    fewshow_vectordb = FAISS.from_texts(to_vectorize, embedding=get_embedding(), metadatas=trainset)
    
    # FAISS DB 저장
    save_faiss_db(fewshow_vectordb, db_path)
    
    return fewshow_vectordb

def save_faiss_db(db, db_path):
    db.save_local(db_path)
    with open(db_path + '_metadata.pkl', 'wb') as f:
        pickle.dump(db, f)

def load_faiss_db(db_path):
    db = FAISS.load_local(db_path, embeddings=get_embedding(), allow_dangerous_deserialization=True)
    with open(db_path + '_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    return db, metadata

def load_chunks_make_docdb(pdf_directory, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    print("Loading PDF files from:", pdf_directory)
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf').replace('\\', '/'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)
    
    # 분할된 텍스트를 벡터로 변환하여 FAISS DB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    print("Done.", len(chunks), "chunks")
    
    print("Creating FAISS DB")
    # FAISS DB 생성 및 저장
    db = FAISS.from_documents(chunks, embedding=get_embedding())
    save_faiss_db(db, db_path)
    print("Done.")
    
    return db

In [5]:
fewshot_vectordb = load_and_vectorize('train.csv', './fewshot_faiss_db')
# save_faiss_db(fewshot_vectordb, 'train_faiss_db')

# Load and index documents from both PDF and text files
train_vectordb = load_chunks_make_docdb('./train_source', './train_faiss_db')
# save_faiss_db(test_vectordb, './train_faiss_db')

# Load and index documents from both PDF and text files
# save_faiss_db(train_vectordb, './train_faiss_db')
test_vectordb = load_chunks_make_docdb('./test_source','./test_faiss_db')

Loading FAISS DB from: ./fewshot_faiss_db


TypeError: FAISS.__init__() missing 4 required positional arguments: 'embedding_function', 'index', 'docstore', and 'index_to_docstore_id'

In [29]:
train_df = pd.read_csv('train.csv')
train_df.drop('SAMPLE_ID', axis=1, inplace=True)
trainset = train_df.to_dict(orient='records')
trainset[1]

{'Source': '1-1 2024 주요 재정통계 1권',
 'Source_path': './train_source/1-1 2024 주요 재정통계 1권.pdf',
 'Question': '2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?',
 'Answer': '2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.'}

In [30]:
test_df = pd.read_csv('test.csv')
test_df.drop('SAMPLE_ID', axis=1, inplace=True)
testset = test_df.to_dict(orient='records')
testset[1]

{'Source': '중소벤처기업부_혁신창업사업화자금(융자)',
 'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf',
 'Question': '중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?'}

In [31]:
fewshot_vectordb,_ = load_faiss_db('./fewshot_faiss_db')
train_vectordb,_ = load_faiss_db('./train_faiss_db')
test_vectordb,_ = load_faiss_db('./test_faiss_db')

# Semantic Similarity Example Selector 설정
example_prompt = PromptTemplate.from_template("Question: {Question}\nAnswer: {Answer}\nSource: {Source}")
# print(example_prompt.invoke(trainset[1]).to_string())

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=fewshot_vectordb,
    k=3,
)

# FewShotPromptTemplate 생성
fewshot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

# 프롬프트 사용 예제
test_input = testset[1]['Question']
ex_qa = fewshot_prompt.invoke({"input": test_input}).to_string()
print(ex_qa)


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Question: 창업사업화지원의 사업목적은 무엇인가?
Answer: 창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.
Source: 중소벤처기업부_창업사업화지원

Question: 언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?
Answer: 기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다.
Source: 중소벤처기업부_창업사업화지원

Question: 2024년 중소벤처기업부의 창업사업화지원 사업에서 어떤 기관들이 사업시행주체로 참여하는가?
Answer: 중소벤처기업부, 창업진흥원이 사업시행주체이다.
Source: 중소벤처기업부_창업사업화지원

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


In [32]:
print(trainset[1])
train_retriever = train_vectordb.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})
retrieved_docs = train_retriever.invoke(testset[1]['Question'])

print(len(retrieved_docs))
retrieved_docs[0].page_content

fewshot 예제

In [33]:
ex_qa = fewshot_prompt.invoke({"input": testset[1]['Question']}).to_string()
fewshot_list = ex_qa.split('\n\n')[:-1]

for i, entry in enumerate(fewshot_list):
    entry_dict = dict([entry.split(': ') for entry in entry.split('\n')])
    retrieved_docs = train_retriever.invoke(entry_dict['Question'])
    entry_dict['context'] = retrieved_docs[0].page_content
    fewshot_list[i] = entry_dict
    
examples = []
for entry in fewshot_list:
    question = entry['Question']
    answer = entry['Answer']
    source = entry['Source']
    context_str = entry['context']
    example_str = f"Question: {question}\nAnswer: {answer}\nSource: {source}\nContext: {context_str}"
    examples.append(example_str)
    
# examples 리스트를 하나의 문자열로 결합
examples_str = "\n\n".join(examples)
example_str

In [34]:
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

full_template = """
너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
반드시 context를 참고하여 답변을 작성해야해.
예시를 참고하여 아래 질문에 대한 답변을 작성해주세요.

이건 질문과 관련된 예시야.
{example}

이 context를 참고하여 아래 질문에 대한 답변을 작성해주세요.
{context}

질문은 이것이다.
{input}"""
prompt = PromptTemplate.from_template(full_template)

In [35]:
def setup_llm_pipeline(model_id):

    model_id = model_id

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf


In [36]:
def setup_llm_pipeline(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)
    hf = HuggingFacePipeline(pipeline=pipe)
    return hf

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 6/6 [01:21<00:00, 13.57s/it]


In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

train_retriever = train_vectordb.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})
test_retriver = test_vectordb.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})

In [ ]:

    for i in tqdm(range(len(testset))):
        ex_qa = fewshot_prompt.invoke({"input": testset[i]['Question']}).to_string()
        fewshot_list = ex_qa.split('\n\n')[:-1]
        for i, entry in enumerate(fewshot_list):
            entry_dict = dict([entry.split(': ') for entry in entry.split('\n')])
            retrieved_docs = train_retriever.invoke(entry_dict['Question'])
            entry_dict['context'] = retrieved_docs[0].page_content
            fewshot_list[i] = entry_dict
            
        examples = []
        for entry in fewshot_list:
            question = entry['Question']
            answer = entry['Answer']
            source = entry['Source']
            context_str = entry['context']
            example_str = f"Question: {question}\nAnswer: {answer}\nSource: {source}\nContext: {context_str}"
            examples.append(example_str)
            
        # examples 리스트를 하나의 문자열로 결합
        examples_str = "\n\n".join(examples)
        full_template = """
        너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
        너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
        반드시 context를 참고하여 답변을 작성해야해.
        예시를 참고하여 아래 질문에 대한 답변을 작성해주세요.
        """ +f"""
        이건 질문과 관련된 예시야.
        {examples_str}
        """+"""
        이 context를 참고하여 아래 질문에 대한 답변을 작성해주세요.
        {context}

        질문은 이것이다.
        {input}"""
        
        prompt = PromptTemplate.from_template(full_template)
        qa_chain = (
        {
            "context": test_retriver | format_docs,
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )
        


  0%|          | 0/98 [00:00<?, ?it/s]c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)